In [ ]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

In [ ]:
import datetime
import os
from typing import Any

import requests
from azure.storage.blob import (
    BlobClient,
    BlobSasPermissions,
    ContainerClient,
    ContainerSasPermissions,
    generate_blob_sas,
    generate_container_sas,
)
from neulander_core.schema.core import AzureBlobDocIn

In [ ]:
account_url = "http://127.0.0.1:10000/devstoreaccount1/"
account_key = os.getenv("ACCOUNT_KEY")

In [ ]:
class NeuAzBlobOperator:
    def __init__(self, account_url: str, account_key: str):
        self.account_url = account_url
        self.account_key = account_key
        self.src_container_client = ContainerClient(
            account_url=self.account_url, container_name="raw"
        )
        self.dest_container_client = ContainerClient(
            account_url=self.account_url, container_name="processed"
        )

    def create_service_sas_blob(self, blob_client: BlobClient):
        # Create a SAS token that's valid for one day, as an example
        start_time = datetime.datetime.now(datetime.timezone.utc)
        expiry_time = start_time + datetime.timedelta(days=1)

        sas_token = generate_blob_sas(
            account_name=blob_client.account_name,
            container_name=blob_client.container_name,
            blob_name=blob_client.blob_name,
            account_key=self.account_key,
            permission=BlobSasPermissions(read=True),
            expiry=expiry_time,
            start=start_time,
        )

        return sas_token

    def create_blob_sas_url(self, blob_name: str):
        blob_client = BlobClient(
            account_url=self.account_url, container_name="raw", blob_name=blob_name
        )

        sas_token = self.create_service_sas_blob(blob_client)

        sas_url = f"{blob_client.url}?{sas_token}"

        return sas_url

    def create_service_sas_container(self, container_client: ContainerClient):
        # Create a SAS token that's valid for one day, as an example
        start_time = datetime.datetime.now(datetime.timezone.utc)
        expiry_time = start_time + datetime.timedelta(days=1)

        sas_token = generate_container_sas(
            account_name=container_client.account_name,
            container_name=container_client.container_name,
            account_key=self.account_key,
            permission=ContainerSasPermissions(write=True, read=True),
            expiry=expiry_time,
            start=start_time,
        )

        return sas_token

    def create_container_sas_url(self, container_client: ContainerClient):
        sas_token = self.create_service_sas_container(container_client)

        sas_url = f"{container_client.url}?{sas_token}"

        return sas_url

In [ ]:
neulander_api_url = "http://127.0.0.1:8000/medcat/1.4"

In [ ]:
nabo = NeuAzBlobOperator(account_key=account_key, account_url=account_url)

In [ ]:
src_container_url = nabo.create_container_sas_url(nabo.src_container_client)
dest_container_url = nabo.create_container_sas_url(nabo.dest_container_client)

## Add some dummy data to the raw container

## Submit jobs to the API

In [ ]:
headers = {"X-API-Key": os.getenv("API_KEY")}

In [ ]:
for i in range(50):
    n = i % 100
    docin = AzureBlobDocIn(
        docid=f"text_{n:03d}",
        docext="txt",
        src=src_container_url,
        dest=dest_container_url,
        docmeta="test",
    )
    r = requests.post(neulander_api_url, data=docin.model_dump_json(), headers=headers)
    print(r.status_code)

In [ ]:
class AzureBlobStorage:
    def __init__(self, sas_url):
        self.sas_url = sas_url
        self.container_client = ContainerClient.from_container_url(sas_url)

    async def read(self, blob_name: str) -> bytes:
        blob_client = self.container_client.get_blob_client(blob_name)
        blob = blob_client.download_blob(timeout=60)
        blob_content = blob.readall()
        return blob_content

    async def write(self, blob_name: str, data: Any):
        now = datetime.now()
        blob_path = f"{now.year}/{now.month}/{now.day}/{blob_name}"
        response = self.container_client.upload_blob(
            name=blob_path, data=data, overwrite=True
        )
        return response